# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [2]:
%pip install --upgrade --quiet langchain langchain-community langchain_openai

Note: you may need to restart the kernel to use updated packages.


If you'd like to use LangSmith, uncomment the below:

In [3]:
import getpass
import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [4]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")

C:\Users\Denish\AppData\Local\Temp\ipykernel_5796\1427064109.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="phi3")



#  How to Install and Run Ollama with the Phi-3 Model

This guide walks you through installing **Ollama** and running the **Phi-3** model on Windows, macOS, and Linux.

---

## Windows

1. **Download Ollama for Windows**  
   Go to: [https://ollama.com/download](https://ollama.com/download)  
   Download and run the installer.

2. **Verify Installation**  
   Open **Command Prompt** and type:
   ```bash
   ollama --version
   ```

3. **Run the Phi-3 Model**  
   In the same terminal:
   ```bash
   ollama run phi3
   ```

4. **If you get a CUDA error (GPU memory issue)**  
   Run Ollama in **CPU mode**:
   ```bash
   set OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  macOS

1. **Install via Homebrew**  
   Open the Terminal and run:
   ```bash
   brew install ollama
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Linux

1. **Install Ollama**  
   Open a terminal and run:
   ```bash
   curl -fsSL https://ollama.com/install.sh | sh
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Notes

- The first time you run `ollama run phi3`, it will **download the model**, so make sure you’re connected to the internet.
- Once downloaded, it works **offline**.
- Keep the terminal open and running in the background while using Ollama from your code or notebook.


## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [5]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [6]:
multiply.invoke({"x": 4, "y": 5})

20.0

In [7]:
@tool
def subtract(x: float, y: float) -> float:
    """Subtracts the second number from the first."""
    return x - y

In [8]:
@tool
def is_prime(number: int) -> bool:
    """Checks if a given integer is a prime number."""
    if number < 2:
        return False
    for i in range(2, int(number**0.5) + 1):
        if number % i == 0:
            return False
    return True

In [9]:
@tool
def concatenate_strings(str1: str, str2: str) -> str:
    """Concatenates two strings together."""
    return str1 + str2

In [10]:
tools = [subtract, is_prime, concatenate_strings]

In [11]:
tools

[StructuredTool(name='subtract', description='Subtracts the second number from the first.', args_schema=<class 'langchain_core.utils.pydantic.subtract'>, func=<function subtract at 0x0000020653C51120>),
 StructuredTool(name='is_prime', description='Checks if a given integer is a prime number.', args_schema=<class 'langchain_core.utils.pydantic.is_prime'>, func=<function is_prime at 0x0000020653C51B20>),
 StructuredTool(name='concatenate_strings', description='Concatenates two strings together.', args_schema=<class 'langchain_core.utils.pydantic.concatenate_strings'>, func=<function concatenate_strings at 0x0000020655A43B00>)]

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [12]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

subtract(x: float, y: float) -> float - Subtracts the second number from the first.
is_prime(number: int) -> bool - Checks if a given integer is a prime number.
concatenate_strings(str1: str, str2: str) -> str - Concatenates two strings together.


In [13]:


system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)



In [14]:
chain = prompt | model
message = chain.invoke({"input": "3 is a prime number?"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)   

```json
{
  "name": "is_prime",
  "arguments": {
    "number": 3
  }
}
```


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [15]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "2 is a prime number?"})

{'name': 'is_prime', 'arguments': {'number': 2}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [28]:
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig

class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""
    name: str
    arguments: Dict[str, Any]

def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function to perform a tool invocation."""
    
    # Mapping from tool name to tool object
    tool_name_to_tool = {tool.name: tool for tool in tools}

    name = tool_call_request["name"]

    # ✅ Add this block to catch unknown tools
    if name not in tool_name_to_tool:
        available = ', '.join(tool_name_to_tool.keys())
        raise ValueError(f"Unknown tool requested: '{name}'. Available tools: {available}")

    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)


Let's test this out 🧪!

In [29]:
invoke_tool({"name": "is_prime", "arguments": {"number": 5}})
#invoke_tool({"name": "concatenate_strings", "arguments": {"str1": "Hello, ", "str2": "world!"}})


True

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [31]:
chain = prompt | model | JsonOutputParser() | invoke_tool
message = chain.invoke({"input": "Is 7 a prime number?"})       



## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [32]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "two strings 'Hello' and 'World' concatenated?"})

{'name': 'concatenate_strings',
 'arguments': {'str1': 'Hello', 'str2': 'World'},
 'output': 'HelloWorld'}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

In [33]:
user_input = "what's subtraction of 100 and 45?"

# Step 1: Run prompt + model + parser (but not invoke_tool yet)
intermediate_chain = prompt | model | JsonOutputParser()
tool_call_request = intermediate_chain.invoke({"input": user_input})

# Step 2: Try to invoke the tool
try:
    result = invoke_tool(tool_call_request)
except Exception as e:
    # Step 3: If it fails, ask the model to revise its output
    correction_prompt = (
        f"The tool call failed with the following error:\n{str(e)}\n"
        f"Please revise the tool call output accordingly."
    )

    # Combine original input with correction prompt
    revised_input = user_input + "\n\n" + correction_prompt
    revised_request = intermediate_chain.invoke({"input": revised_input})

    # Retry with revised tool call
    result = invoke_tool(revised_request)

# Step 4: Use or return result
print(result)


55.0
